In [1]:
import pandas as pd
import numpy  as np

In [2]:
data = pd.read_csv('Crime_Data_from_2020_to_Present.csv', dtype=str, na_values='')

In [3]:
df = pd.DataFrame(data)

In [4]:
df.shape

(1005199, 28)

In [5]:
print(df.dtypes)

DR_NO             object
Date Rptd         object
DATE OCC          object
TIME OCC          object
AREA              object
AREA NAME         object
Rpt Dist No       object
Part 1-2          object
Crm Cd            object
Crm Cd Desc       object
Mocodes           object
Vict Age          object
Vict Sex          object
Vict Descent      object
Premis Cd         object
Premis Desc       object
Weapon Used Cd    object
Weapon Desc       object
Status            object
Status Desc       object
Crm Cd 1          object
Crm Cd 2          object
Crm Cd 3          object
Crm Cd 4          object
LOCATION          object
Cross Street      object
LAT               object
LON               object
dtype: object


In [6]:
df['AREA'].value_counts(dropna=False).sort_index()

AREA
01    69673
02    46826
03    57512
04    37097
05    41462
06    52430
07    48240
08    45731
09    42883
10    42157
11    42962
12    61762
13    49181
14    59521
15    51107
16    33136
17    41766
18    49941
19    40365
20    50071
21    41376
Name: count, dtype: int64

### Lookup Tables

In [7]:
area = df[['AREA', 'AREA NAME']].drop_duplicates().sort_values(by='AREA').dropna(how='all').reset_index(drop=True)

In [8]:
area

,AREA,AREA NAME
0,01,Central
1,02,Rampart
2,03,Southwest
3,04,Hollenbeck
4,05,Harbor
5,06,Hollywood
6,07,Wilshire
7,08,West LA
8,09,Van Nuys
9,10,West Valley


In [9]:
premis = df[['Premis Cd', 'Premis Desc']].drop_duplicates().sort_values(by='Premis Cd').dropna(how='all').reset_index(drop=True)

In [10]:
premis

,Premis Cd,Premis Desc
0,101,STREET
1,102,SIDEWALK
2,103,ALLEY
3,104,DRIVEWAY
4,105,PEDESTRIAN OVERCROSSING
...,...,...
309,972,NaN
310,973,NaN
311,974,NaN
312,975,NaN


In [11]:
weapon = df[['Weapon Used Cd', 'Weapon Desc']].drop_duplicates().sort_values(by='Weapon Used Cd').dropna(how='all').reset_index(drop=True)

In [12]:
weapon

,Weapon Used Cd,Weapon Desc
0,101,REVOLVER
1,102,HAND GUN
2,103,RIFLE
3,104,SHOTGUN
4,105,SAWED OFF RIFLE/SHOTGUN
...,...,...
74,512,MACE/PEPPER SPRAY
75,513,STUN GUN
76,514,TIRE IRON
77,515,PHYSICAL PRESENCE


In [13]:
status = df[['Status', 'Status Desc']].drop_duplicates().sort_values(by='Status').dropna(how='all').reset_index(drop=True)

In [14]:
status

,Status,Status Desc
0,AA,Adult Arrest
1,AO,Adult Other
2,CC,UNK
3,IC,Invest Cont
4,JA,Juv Arrest
5,JO,Juv Other
6,NaN,UNK


In [15]:
crm_data = df[['Crm Cd', 'Crm Cd Desc', 'Part 1-2']].drop_duplicates().sort_values(by='Crm Cd').dropna(how='all').reset_index(drop=True)

In [16]:
crm_data

,Crm Cd,Crm Cd Desc,Part 1-2
0,110,CRIMINAL HOMICIDE,1
1,113,"MANSLAUGHTER, NEGLIGENT",1
2,121,"RAPE, FORCIBLE",1
3,122,"RAPE, ATTEMPTED",1
4,210,ROBBERY,1
...,...,...,...
135,949,ILLEGAL DUMPING,2
136,950,"DEFRAUDING INNKEEPER/THEFT OF SERVICES, OVER $...",2
137,951,"DEFRAUDING INNKEEPER/THEFT OF SERVICES, $950 &...",2
138,954,CONTRIBUTING,2


### Combine Crm Cd 1-4

In [17]:
combined_crms = pd.concat([df['Crm Cd 1'], df['Crm Cd 2'], df['Crm Cd 3'],df['Crm Cd 4']]).drop_duplicates().dropna(how='all').sort_values().reset_index(drop=True)

In [18]:
crm_levels = pd.DataFrame(combined_crms, columns=['Crm Cd'])

### Left merge crime from levels and crime codes

In [19]:
crm = pd.merge(crm_levels, crm_data, how='left', on='Crm Cd').replace(np.nan, None)

In [20]:
crm

,Crm Cd,Crm Cd Desc,Part 1-2
0,110,CRIMINAL HOMICIDE,1
1,113,"MANSLAUGHTER, NEGLIGENT",1
2,121,"RAPE, FORCIBLE",1
3,122,"RAPE, ATTEMPTED",1
4,210,ROBBERY,1
...,...,...,...
153,994,None,None
154,996,None,None
155,997,None,None
156,998,None,None


### Save Data Source

In [21]:
area.to_csv('_area.csv', index=False)

In [22]:
premis.to_csv('_premis.csv', index=False)

In [23]:
weapon.to_csv('_weapon.csv', index=False)

In [24]:
status.to_csv('_status.csv', index=False)

In [25]:
crm.to_csv('_crm.csv', index=False)

### Insert Scripts

In [26]:
def inserts(file, table):
    data = pd.read_csv(file)
    insert_values = []
    for index, row in data.iterrows():
        values = []
        for attribute in row:
            att_val = "NULL" if pd.isna(attribute) else '"'+str(attribute)+'"'
            values.append(att_val)
        attribute_values = "(" + ','.join(values) + ")";
        insert_values.append(attribute_values);
        cols = ','.join(data.columns).replace(' ', '_').replace('-', '_')
    values = ','.join(insert_values)
    sql = "INSERT INTO " + table + " (" + cols + ") VALUES " + values + ";";
    filename = 'insert_' + table + '.sql'
    with open(filename, "w") as file:
        file.write(sql)

In [27]:
inserts('_area.csv', 'Area')

In [31]:
inserts('_crm.csv', 'Crime')

In [32]:
inserts('_premis.csv', 'Premis')

In [33]:
inserts('_weapon.csv', 'Weapon')

In [34]:
inserts('_status.csv', 'Status')

In [35]:
inserts('mocodes.csv', 'Mocodes')